In [1]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
training_data_len = math.ceil(len(label)* 0.8)

df = pd.read_pickle("sp500.pkl")
df_scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = df_scaler.fit_transform(df)

label = df["close"].values
label_scaler = MinMaxScaler(feature_range=(0,1))
scaled_label = label_scaler.fit_transform(label)

In [9]:
df.columns

Index(['open', 'high', 'low', 'close', 'adj close', 'volume', 'change',
       'open_delta', 'close_delta', 'volume_delta', 'high_low_diff_ratio',
       'open_close_diff_ratio', 'close_-2_r', 'close_-6_r', 'kdjk', 'kdjd',
       'kdjj', 'macd', 'macds', 'macdh', 'rsi_6', 'rsi_12', 'wr_6', 'wr_12',
       'cci', 'atr', 'dma', 'vr', 'boll_-1_d', 'boll_ub_-1_d', 'boll_lb_-1_d',
       'boll_k_diff', 'high_low_diff', 'open_close_diff'],
      dtype='object')

# Model

In [15]:
from tensorflow import keras
model = keras.models.load_model('./models/best_model.h5')

# Construct Data for LSTM 

In [18]:
# Train data
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(30, len(train_data)):
    x_train.append(train_data[i-30:i, ])
    y_train.append(train_data[i, ])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

In [19]:
# Test Data
test_data = scaled_data[training_data_len-30: , : ]
x_test = []
y_test = []
for i in range(30, len(test_data)):
    x_test.append(test_data[i-30: i, ])
    y_test.append(test_data[i, ])

x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))

In [29]:
predicted_train = model.predict(x_train)
train_label = scaled_label[30:x_train.shape[0]+30]
predicted_test = model.predict(x_test)
test_label = scaled_label[x_train.shape[0]+30:]

In [30]:
print(predicted_train.shape)
print(predicted_test.shape)
print(train_label.shape)
print(test_label.shape)

(1982, 34)
(503, 34)
(1982, 34)
(503, 34)


# As function

In [46]:
def getScaledData(file):
    df = pd.read_pickle(file)
    df_scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = df_scaler.fit_transform(df)
    
    label = df["close"].values
    label_scaler = MinMaxScaler(feature_range=(0,1))
    scaled_label = label_scaler.fit_transform(label.reshape(-1,1))
    
    return scaled_data, scaled_label

def getPredictionFromLSTM(training_data_len, scaled_data, scaled_label, model):
    # Train data
    train_data = scaled_data[0: training_data_len, :]
    x_train = []
    y_train = []
    for i in range(30, len(train_data)):
        x_train.append(train_data[i-30:i, ])
        y_train.append(train_data[i, ])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))
    
    # Test Data
    test_data = scaled_data[training_data_len-30: , : ]
    x_test = []
    y_test = []
    for i in range(30, len(test_data)):
        x_test.append(test_data[i-30: i, ])
        y_test.append(test_data[i, ])
    x_test, y_test = np.array(x_test), np.array(y_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], x_test.shape[2]))
    
    predicted_train = model.predict(x_train)
    train_label = scaled_label[30:x_train.shape[0]+30]
    predicted_test = model.predict(x_test)
    test_label = scaled_label[x_train.shape[0]+30:]
    
    return predicted_train, train_label.reshape(-1), predicted_test, test_label.reshape(-1)

In [47]:
scaled_data, scaled_label = getScaledData('./sp500.pkl')
model = keras.models.load_model('./models/best_model.h5')
training_data_len = math.ceil(len(label)* 0.8)
predicted_train, train_label, predicted_test, test_label = getPredictionFromLSTM(training_data_len, scaled_data, scaled_label, model)

In [48]:
print(predicted_train.shape)
print(predicted_test.shape)
print(train_label.shape)
print(test_label.shape)

(1982, 34)
(503, 34)
(1982,)
(503,)
